# Введение в обработку естественного языка

## Урок 9. Языковое моделирование

**Задание:**
Разобраться с моделью генерации текста, собрать самим или взять датасет с вебинара и обучить генератор


In [1]:
import numpy as np
import os
import time
import tensorflow as tf

In [2]:
path_to_file = 'evgenyi_onegin.txt'

In [3]:
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print('Length of text: {} characters'.format(len(text)))

Length of text: 286984 characters


In [4]:
print(text[:500])

Александр Сергеевич Пушкин

                                Евгений Онегин
                                Роман в стихах

                        Не мысля гордый свет забавить,
                        Вниманье дружбы возлюбя,
                        Хотел бы я тебе представить
                        Залог достойнее тебя,
                        Достойнее души прекрасной,
                        Святой исполненной мечты,
                        Поэзии живой и ясной,
                        Высо


In [5]:
text = text + text

In [6]:
# The unique characters in the file
vocab = sorted(set(text))
print('{} unique characters'.format(len(vocab)))

131 unique characters


In [7]:
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])

In [8]:
print(text_as_int, text[:5000], len(text_as_int), len(text), sep='\n')

[ 71 110 104 ... 104 121   0]
Александр Сергеевич Пушкин

                                Евгений Онегин
                                Роман в стихах

                        Не мысля гордый свет забавить,
                        Вниманье дружбы возлюбя,
                        Хотел бы я тебе представить
                        Залог достойнее тебя,
                        Достойнее души прекрасной,
                        Святой исполненной мечты,
                        Поэзии живой и ясной,
                        Высоких дум и простоты;
                        Но так и быть - рукой пристрастной
                        Прими собранье пестрых глав,
                        Полусмешных, полупечальных,
                        Простонародных, идеальных,
                        Небрежный плод моих забав,
                        Бессонниц, легких вдохновений,
                        Незрелых и увядших лет,
                        Ума холодных наблюдений
                        И сердца 

### train and target

In [9]:
# The maximum length sentence you want for a single input in characters
seq_length = 100
examples_per_epoch = len(text)//(seq_length+1)

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(5):
    print(idx2char[i.numpy()])

А
л
е
к
с


In [10]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

for item in sequences.take(5):
    print(repr(''.join(idx2char[item.numpy()])))

'Александр Сергеевич Пушкин\n\n                                Евгений Онегин\n                          '
'      Роман в стихах\n\n                        Не мысля гордый свет забавить,\n                        '
'Вниманье дружбы возлюбя,\n                        Хотел бы я тебе представить\n                        '
'Залог достойнее тебя,\n                        Достойнее души прекрасной,\n                        Свят'
'ой исполненной мечты,\n                        Поэзии живой и ясной,\n                        Высоких д'


In [11]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

Print the first example input and target values:

In [12]:
for input_example, target_example in  dataset.take(1):
    print('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
    print('Target data:', repr(''.join(idx2char[target_example.numpy()])))

Input data:  'Александр Сергеевич Пушкин\n\n                                Евгений Онегин\n                         '
Target data: 'лександр Сергеевич Пушкин\n\n                                Евгений Онегин\n                          '


In [13]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

<_BatchDataset element_spec=(TensorSpec(shape=(64, 100), dtype=tf.int32, name=None), TensorSpec(shape=(64, 100), dtype=tf.int32, name=None))>

In [14]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 128

# Number of RNN units
rnn_units = 1024

In [15]:
model = tf.keras.Sequential(
    [
        tf.keras.layers.Embedding(vocab_size, embedding_dim),
        tf.keras.layers.LSTM(rnn_units, return_sequences=True),
        tf.keras.layers.Dense(vocab_size)
    ]
)

In [16]:
class RNNgenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, batch_size):
        super(RNNgenerator, self).__init__()
        
        self.emb = tf.keras.layers.Embedding(vocab_size, embedding_dim)
                                 
        self.gru1 = tf.keras.layers.GRU(rnn_units,
                            return_sequences=True,
                            recurrent_initializer='glorot_uniform')
        self.gru2 = tf.keras.layers.GRU(rnn_units,
                            return_sequences=True,
                            recurrent_initializer='glorot_uniform')
                           
        self.fc = tf.keras.layers.Dense(vocab_size)

    def call(self, x):
        emb_x = self.emb(x)
        x1 = self.gru1(emb_x)
        x = x1
        for _ in range(3):
            x = self.gru2(x)
        #x = self.gru1(x)
        x = (x + x1)/2
        return self.fc(x)

model = RNNgenerator(vocab_size, embedding_dim, BATCH_SIZE)

In [17]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dim,
                                  batch_input_shape=[batch_size, None]),
                                 
        tf.keras.layers.LSTM(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),

        tf.keras.layers.LSTM(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),

         tf.keras.layers.LSTM(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),
        
        tf.keras.layers.LSTM(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),
                                   
        tf.keras.layers.Dense(vocab_size)
    ])
    return model

In [18]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dim),
                                 
        tf.keras.layers.GRU(rnn_units,
                            return_sequences=True,
                            stateful=False,
                            recurrent_initializer='glorot_uniform'),

        tf.keras.layers.GRU(rnn_units,
                            return_sequences=True,
                            stateful=False,
                            recurrent_initializer='glorot_uniform'),

         tf.keras.layers.GRU(rnn_units,
                            return_sequences=True,
                            stateful=False,
                            recurrent_initializer='glorot_uniform'),
                                   
        tf.keras.layers.Dense(vocab_size)
    ])
    return model

In [19]:
model = build_model(
    vocab_size=len(vocab),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units,
    batch_size=BATCH_SIZE)

In [20]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 131) # (batch_size, sequence_length, vocab_size)


In [21]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, None, 128)         16768     
                                                                 
 gru_2 (GRU)                 (None, None, 1024)        3545088   
                                                                 
 gru_3 (GRU)                 (None, None, 1024)        6297600   
                                                                 
 gru_4 (GRU)                 (None, None, 1024)        6297600   
                                                                 
 dense_2 (Dense)             (None, None, 131)         134275    
                                                                 
Total params: 16291331 (62.15 MB)
Trainable params: 16291331 (62.15 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [22]:
example_batch_predictions[0]

<tf.Tensor: shape=(100, 131), dtype=float32, numpy=
array([[ 1.4480156e-03,  6.1347143e-04, -2.7535041e-04, ...,
        -2.6652848e-04, -1.0005252e-03,  3.2568153e-04],
       [ 3.5887589e-03,  1.4901013e-03, -7.8802003e-04, ...,
        -9.0823229e-04, -2.3365221e-03,  8.4530719e-04],
       [ 5.7255575e-03,  2.3432733e-03, -1.4728258e-03, ...,
        -1.7652309e-03, -3.5244189e-03,  1.3624423e-03],
       ...,
       [-9.8530014e-05,  1.6403088e-04,  2.9769673e-03, ...,
         3.1422960e-04, -1.0240009e-03,  1.3621586e-03],
       [ 4.1898387e-04,  3.5873812e-04,  2.8295584e-03, ...,
         1.3159723e-03, -2.0471199e-03,  2.2264889e-03],
       [ 2.3799846e-03,  1.0269865e-03,  2.1000812e-03, ...,
         1.4832973e-03, -3.5632013e-03,  3.0405051e-03]], dtype=float32)>

In [23]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

In [24]:
print("Input: \n", repr("".join(idx2char[input_example_batch[0]])))
print()
print("Next Char Predictions: \n", repr("".join(idx2char[sampled_indices ])))

Input: 
 '                       Теснились в душу глубоко.\n                        Он меж печатными строками\n '

Next Char Predictions: 
 'Чю.h WChmЖР8оYалИ\nщGДп9АbОCиsЯД7ВюSEi)яs}щDDЧЯчoНжhВьp6ЕR"Af РF\nдсI!"TB0ЯЭ;6ъQэ7О5b2ЗкHьpя:ЕЧ,о)YУБ{'


## Train the model

In [25]:
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("scalar_loss:      ", example_batch_loss.numpy().mean())

Prediction shape:  (64, 100, 131)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       4.873134


In [26]:
model.compile(optimizer='adam', loss=loss)

### Configure checkpoints

In [27]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_freq=88*3,
    save_weights_only=True)

### Execute the training

In [28]:
EPOCHS = 20

In [29]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/20
88/88 [==============================] - 915s 10s/step - loss: 2.0044
Epoch 2/20
88/88 [==============================] - 910s 10s/step - loss: 1.4499
Epoch 3/20
88/88 [==============================] - 932s 11s/step - loss: 1.3110
Epoch 4/20
88/88 [==============================] - 997s 11s/step - loss: 1.2218
Epoch 5/20
88/88 [==============================] - 1016s 12s/step - loss: 1.1354
Epoch 6/20
88/88 [==============================] - 1019s 12s/step - loss: 1.0439
Epoch 7/20
88/88 [==============================] - 1018s 12s/step - loss: 0.9497
Epoch 8/20
88/88 [==============================] - 997s 11s/step - loss: 0.9067
Epoch 9/20
88/88 [==============================] - 1029s 12s/step - loss: 0.7945
Epoch 10/20
88/88 [==============================] - 1024s 12s/step - loss: 0.6966
Epoch 11/20
88/88 [==============================] - 1031s 12s/step - loss: 0.5999
Epoch 12/20
88/88 [==============================] - 1012s 11s/step - loss: 0.5000
Epoch 13/20
88/88 

## Generate text

In [30]:
tf.train.latest_checkpoint(checkpoint_dir)

'./training_checkpoints\\ckpt_18'

In [31]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1, None]))

In [32]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, None, 128)         16768     
                                                                 
 gru_5 (GRU)                 (None, None, 1024)        3545088   
                                                                 
 gru_6 (GRU)                 (None, None, 1024)        6297600   
                                                                 
 gru_7 (GRU)                 (None, None, 1024)        6297600   
                                                                 
 dense_3 (Dense)             (None, None, 131)         134275    
                                                                 
Total params: 16291331 (62.15 MB)
Trainable params: 16291331 (62.15 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [33]:
def generate_text(model, start_string):
    # Evaluation step (generating text using the learned model)

    # Number of characters to generate
    num_generate = 500

    # Converting our start string to numbers (vectorizing)
    input_eval = [char2idx[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)

    # Empty string to store our results
    text_generated = []

    # Low temperature results in more predictable text.
    # Higher temperature results in more surprising text.
    # Experiment to find the best setting.
    temperature = 0.5

    # Here batch size == 1
    model.reset_states()
    for i in range(num_generate):
        predictions = model(input_eval)
        predictions = tf.squeeze(predictions, 0)
        # using a categorical distribution to predict the character returned by the model
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1, 0].numpy()

        # Pass the predicted character as the next input to the model
        # along with the previous hidden state
        input_eval = tf.expand_dims([predicted_id], 0)

        text_generated.append(idx2char[predicted_id])

    return (start_string + ''.join(text_generated))

In [34]:
text_ = generate_text(model, start_string=u"И вот идет уже ")
print(text_)

И вот идет уже бый,                                                                                                                                                                                                           И                                                              виче                                                                                                                                                                                                    Тазде                         


In [35]:
len(text_)

515